In [ ]:
!nvidia-smi

## Dependencies

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

!pip install --no-deps -U ../input/pytorch-image-models/
!pip install --no-deps -U ../input/effdet-latestvinbigdata-wbf-fused/omegaconf-2.0.6-py3-none-any.whl
!pip install --no-deps -U ../input/effdet-latestvinbigdata-wbf-fused/pycocotools-2.0.2/
!pip install --no-deps -U ../input/efficientdetpytorch/
!pip install --no-deps -U ../input/ensemble-boxes-104/ensemble_boxes-1.0.4
!pip install /kaggle/input/mishcuda/mish-cuda/

!pip install '/kaggle/input/mmdetectionv2140/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmcv_full-1_3_8-cu110-torch1_7_0/mmcv_full-1.3.8-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps

!cp -r /kaggle/input/mmdetectionv2140/mmdetection-2.14.0 /kaggle/working/
!mv /kaggle/working/mmdetection-2.14.0 /kaggle/working/mmdetection
!pip install -e '/kaggle/working/mmdetection/'

## Imports and Env Setup

In [ ]:
import sys, warnings, pandas
pandas.reset_option("all")
warnings.filterwarnings('ignore')
sys.path.append('../input/siimfisabiorsna-covid19-checkpoints/pytorch-toolbelt/pytorch-toolbelt-develop/')
sys.path.append('../input/ttach-pytorch/')
sys.path.append('../input/siim-code/classification_code/')
sys.path.append('./mmdetection/')

In [ ]:
import gc, json, cv2

import albumentations as albu
from albumentations.pytorch import ToTensorV2

from tqdm.auto import tqdm
from fastai.vision.all import *
from fastai.medical.imaging import get_dicom_files

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import ttach as tta
from timm.models import load_checkpoint
from torch.utils.data import DataLoader, Dataset

from src.defaults import get_cfg
from src.models import build_model, create_backbone
from heng_inference_binary_multiaux_final import Net,\
     BenchPredictHengBinary

In [ ]:
df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
if df.shape[0] == 2477:
    fast_sub = True
else:
    fast_sub = False
print(fast_sub)

In [ ]:
filepaths = get_dicom_files("/kaggle/input/siim-covid19-detection/test/", recurse=True)
if fast_sub:
    filepaths = filepaths[:5]
test_df = pd.DataFrame({'filepath': filepaths.map(lambda x: str(x))})
test_df['image_id'] = test_df.filepath.map(lambda x: str(x).split('/')[-1].replace('.dcm', '') + '_image')
test_df['study_id'] = test_df.filepath.map(lambda x: str(x).split('/')[-3].replace('.dcm', '') + '_study')
display_df(test_df.head())

In [ ]:
label2name   = {0: 'atypical', 1: 'indeterminate', 2: 'negative', 3: 'typical'}
class_labels = ['0', '1', '2', '3']

IMG_SIZE  = 1024
TEST_PATH = '/kaggle/tmp/test/'
os.makedirs(TEST_PATH, exist_ok=True)

## Serialization of Test Images

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    if voi_lut: 
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else: 
        data = dicom.pixel_array
        
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data 
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8) 
    return data

def resize(array, size, keep_ratio=True, resample=Image.LANCZOS):
    im = Image.fromarray(array)
    if keep_ratio: 
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    return im

def resize_and_save(file_path):
    base_dir = TEST_PATH
    xray = read_xray(file_path)
    img  = resize(xray, IMG_SIZE, keep_ratio=True)
    filename = file_path.split('/')[-1].split('.')[0]
    img.save(os.path.join(base_dir, f'{filename}.png'))
    return filename.replace('dcm','') + '_image', xray.shape[0], xray.shape[1]

def serialize(paths):
    info = []
    dim0 = []
    dim1 = []
    iterator = tqdm(paths, total=len(paths), dynamic_ncols=True )
    for idx , p in enumerate(iterator): 
        i, d0, d1 = resize_and_save(p)
        info.append(i)  
        dim0.append(d0)
        dim1.append(d1)
    return info, dim0, dim1

In [ ]:
filepaths = test_df.filepath.iloc[:test_df.shape[0]]
(info, dim0, dim1) = serialize(filepaths)

In [ ]:
df = pd.DataFrame({'image_id': list(info)})
df['image_path'] = TEST_PATH + df.image_id.map(lambda x: x.replace('_image','')) + '.png'
df['dim0'] = list(dim0)
df['dim1'] = list(dim1)

test_df = pd.merge(test_df, df, on='image_id', how='left')
test_df.drop(columns=['filepath'], inplace=True)
display_df(test_df.head())

In [ ]:
image_df = test_df.drop(columns=["study_id"], inplace=False)
study_df = test_df.drop(columns=["image_id"], inplace=False)
test_df.loc[:, class_labels] = 0
display_df(test_df.head())

## Study Level Predictions

In [ ]:
class LoadImages(Dataset):
    def __init__(self, df, transform):
        self.file_names = L(list(df.image_path.iloc[:test_df.shape[0]]))
        self.transform  = transform

    def __len__(self): 
        return len(self.file_names)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        assert os.path.exists(file_name)
        image = cv2.imread(file_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transform(image=image)['image']
        return image
    
def pre_processing(sz, *args, **kwargs):
    return albu.Compose([
        albu.Resize(sz, sz), 
        albu.ToFloat(p=1.0, max_value=255.0), 
        ToTensorV2(p=1.0)
    ])

def get_tta_transforms(): 
    return tta.Compose([tta.HorizontalFlip()])


class BenchPredict(nn.Module):
    def __init__(self, model):
        super(BenchPredict, self).__init__()
        self.model = model
    
    def forward(self, input):
        logits = self.model(input)['output']
        return F.softmax(logits)


@torch.no_grad()
def get_preds(model, test_loader):
    batch_probs = []
    for i, (images) in enumerate(tqdm(test_loader, dynamic_ncols=True, leave=False)):
        images = images.cuda()
        y_preds= model(images)
        batch_probs.append(y_preds)
        del y_preds, images
        gc.collect()
        torch.cuda.empty_cache()
    return torch.cat(batch_probs).data.cpu().numpy()

@delegates(load_checkpoint)
def make_model(cfg, state, **kwargs):
    model = build_model(cfg)
    load_checkpoint(model, state, **kwargs)
    bench = BenchPredict(model=model)
    bench.cuda()
    bench.eval()
    return bench

def get_PredictionString(row, df, thr=0.0):
    string = ''
    for idx in list(df.columns[1:]):
        conf =  row[idx]
        if conf>thr:
            string += f'{idx} {conf:.6f} 0 0 1 1 '
    string = string.strip()
    return string

## Study : Mix Model Predictions

In [ ]:
used_models = [
    '../input/studymodelsbest/001cf4cc-8939v2l_noisy_Fine/fold1/model_best.pth.tar',
    '../input/studymodelsbest/001cf4cc-8939v2l_noisy_Fine/fold3/model_best.pth.tar',
    '../input/studymodelsbest/001cf4cc-8939v2l_noisy_Fine/fold4/model_best.pth.tar',
    '../input/studymodelsbest/001cf4cc-8939v2m_noisy640/fold0/model_best.pth.tar',
    '../input/studymodelsbest/7f8543b6-6957eb5_Noisy/fold2/model_best.pth.tar',
    
    '../input/studymodelsbest/7f8543b6-6957eb7_noisy640_Fine/fold3/model_best.pth.tar',
    
    '../input/study-models-best-part2/6957eb7_noisy640_Fine/fold2/model_best.pth.tar',
    '../input/study-models-best-part2/8939v2l_640_rotate/fold1/model_best.pth.tar',
    '../input/study-models-best-part2/8939v2l_noisy512_50_hard/fold3/model_best.pth.tar',
    '../input/study-models-best-part2/8939v2l_noisy_512_rotate_Fine/fold4/model_best.pth.tar',
    '../input/study-models-best-part2/8939v2m_1024_noisy/fold0/model_best.pth.tar',
]
    
configs = [    
    '../input/studymodelsbest/001cf4cc-8939v2l_noisy_Fine/fold1/config.yaml',
    '../input/studymodelsbest/001cf4cc-8939v2l_noisy_Fine/fold3/config.yaml',
    '../input/studymodelsbest/001cf4cc-8939v2l_noisy_Fine/fold4/config.yaml',
    '../input/studymodelsbest/001cf4cc-8939v2m_noisy640/fold0/config.yaml',
    '../input/studymodelsbest/7f8543b6-6957eb5_Noisy/fold2/config.yaml',
    
    '../input/studymodelsbest/7f8543b6-6957eb7_noisy640_Fine/fold3/config.yaml',
    
    '../input/study-models-best-part2/6957eb7_noisy640_Fine/fold2/config.yaml',
    '../input/study-models-best-part2/8939v2l_640_rotate/fold1/config.yaml',
    '../input/study-models-best-part2/8939v2l_noisy512_50_hard/fold3/config.yaml',
    '../input/study-models-best-part2/8939v2l_noisy_512_rotate_Fine/fold4/config.yaml',
    '../input/study-models-best-part2/8939v2m_1024_noisy/fold0/config.yaml'
]

preds = []

for c, chkpt in tqdm(zip(configs, used_models), total=len(used_models), dynamic_ncols=True):
    CFG = get_cfg()
    CFG.merge_from_file(c)
    CFG.MODEL.BACKBONE.PRETRAINED = False

    transform = pre_processing(sz=CFG.INPUT.INPUT_SIZE)
    dset = LoadImages(test_df, transform=transform)
    dl = DataLoader(dset, batch_size=16)
    
    model = make_model(CFG, chkpt)
    ttas = get_tta_transforms()
    model = tta.ClassificationTTAWrapper(model, ttas) 
    
    predict = get_preds(model, dl)
    preds.append(predict)
    
    del model, dl,dset, predict
    gc.collect(); torch.cuda.empty_cache()

## v2m Predictions

In [ ]:
used_models = [
    '../input/siimfisabiorsna-covid19-checkpoints/8f5d9473-4fda/tf_efficientnetv2_m_in21ft1k_pcam_4x_512_auxv1_mish_sam_ns_fold0-8f5d9473-4fda.pth.tar',
    '../input/siimfisabiorsna-covid19-checkpoints/8f5d9473-4fda/tf_efficientnetv2_m_in21ft1k_pcam_4x_512_auxv1_mish_sam_ns_fold1-8f5d9473-4fda.pth.tar',
    '../input/siimfisabiorsna-covid19-checkpoints/8f5d9473-4fda/tf_efficientnetv2_m_in21ft1k_pcam_4x_512_auxv1_mish_sam_ns_fold2-8f5d9473-4fda.pth.tar',
    '../input/siimfisabiorsna-covid19-checkpoints/8f5d9473-4fda/tf_efficientnetv2_m_in21ft1k_pcam_4x_512_auxv1_mish_sam_ns_fold3-8f5d9473-4fda.pth.tar',
    '../input/siimfisabiorsna-covid19-checkpoints/8f5d9473-4fda/tf_efficientnetv2_m_in21ft1k_pcam_4x_512_auxv1_mish_sam_ns_fold4-8f5d9473-4fda.pth.tar',

]
    
configs = [
    '../input/siimfisabiorsna-covid19-checkpoints/8f5d9473-4fda/config.yml',
    '../input/siimfisabiorsna-covid19-checkpoints/8f5d9473-4fda/config.yml',
    '../input/siimfisabiorsna-covid19-checkpoints/8f5d9473-4fda/config.yml',
    '../input/siimfisabiorsna-covid19-checkpoints/8f5d9473-4fda/config.yml',
    '../input/siimfisabiorsna-covid19-checkpoints/8f5d9473-4fda/config.yml'
]

v2m_preds = []

for c, chkpt in tqdm(zip(configs, used_models), total=len(used_models), dynamic_ncols=True):
    CFG = get_cfg()
    CFG.merge_from_file(c)
    CFG.MODEL.BACKBONE.PRETRAINED = False

    transform = pre_processing(sz=CFG.INPUT.INPUT_SIZE)
    dset = LoadImages(test_df, transform=transform)
    dl = DataLoader(dset, batch_size=16)
    
    model = make_model(CFG, chkpt)
    model = tta.ClassificationTTAWrapper(model, get_tta_transforms()) 
    
    predict = get_preds(model, dl)
    v2m_preds.append(predict)
    
    del model, dl,dset, predict
    torch.cuda.empty_cache(); gc.collect()

## Binary Predictions

In [ ]:
@delegates(load_checkpoint)
def load_binary_model(state, **kwargs):
    model = Net()
    load_checkpoint(model, state, **kwargs)
    bench = BenchPredictHengBinary(binary_model=model)
    bench.cuda()
    bench.eval()
    return bench

In [ ]:
used_models = [
    '../input/covid-binary-models/eb6ns_512_binary_multimask_noisy_new/eb6ns_512_binary_multimask_noisy_fold0_1374.pth',
    '../input/covid-binary-models/eb6ns_512_binary_multimask_noisy_new/eb6ns_512_binary_multimask_noisy_fold1_1408_ext.pth',
    '../input/covid-binary-models/eb6ns_512_binary_multimask_noisy_new/eb6ns_512_binary_multimask_noisy_fold2_1376_ext.pth',
    '../input/covid-binary-models/eb6ns_512_binary_multimask_noisy_new/eb6ns_512_binary_multimask_noisy_fold3_1377_ext.pth',
    '../input/covid-binary-models/eb6ns_512_binary_multimask_noisy_new/eb6ns_512_binary_multimask_noisy_fold4_1343.pth',
]

binary_preds = []

for chkpt in tqdm(used_models, total=len(used_models), dynamic_ncols=True):

    transform = pre_processing(sz=512)
    dset = LoadImages(test_df, transform=transform)
    dl   = DataLoader(dset, batch_size=16)

    ttas  = get_tta_transforms() 
    
    model = load_binary_model(chkpt)
    model = tta.ClassificationTTAWrapper(model, ttas) 
    
    predict = get_preds(model, dl)
    binary_preds.append(predict)
    
    del model, dl, dset, predict
    torch.cuda.empty_cache()
    gc.collect()

## Process Study Predictions

In [ ]:
final_mix_preds_study = np.mean(preds, axis=0)
final_v2m_preds_study = np.mean(v2m_preds, axis=0)
study_final_preds = 0.85*final_mix_preds_study + 0.15*final_v2m_preds_study

test_df.loc[:test_df.shape[0], class_labels] = study_final_preds
study_df = test_df.groupby(['study_id'])[class_labels].mean().reset_index()
study_df.rename(columns={'study_id': 'id'}, inplace=True)
study_df.columns = ['id',"atypical", "indeterminate", "negative", "typical"]

study_df['PredictionString'] = study_df.apply(partial(get_PredictionString, df=study_df), axis=1)
study_df.to_csv('./study_df.csv', index=False)
display_df(study_df.head())

## Process Binary Predictions

In [ ]:
# Mean of 5 fold binary model predictions
binary_predictions = np.mean(binary_preds, axis=0)
# Store binary predictions
test_df.loc[:test_df.shape[0], 'none'] = binary_predictions
# Prepare a dataframe with only none predictions
bin_df = test_df[["study_id", "image_id", "none"]]
# Sort values to ensure same indices
bin_df = bin_df.sort_values(by='image_id', ascending=True)
bin_df.head()

In [ ]:
# Mean of 5 fold v2m predictions
class_labels = ["atypical", "indeterminate", "negative", "typical"]
preds2 = np.mean(v2m_preds, axis=0)
test_df.loc[:test_df.shape[0], class_labels] = preds2
# Store v2m predictions
v2m_df = test_df[["study_id", "image_id", "negative"]]
# Prepare a dataframe with only none predictions
v2m_df = v2m_df.rename(columns=dict(negative='none'), inplace=False)
# Sort values to ensure same indices
v2m_df = v2m_df.sort_values(by='image_id', ascending=True)
v2m_df.head()

In [ ]:
binary_submission = v2m_df.copy()
binary_submission['none'] = 0.6*v2m_df['none'].values + 0.4*bin_df['none'].values

binary_submission = binary_submission.reset_index(inplace=False, drop=True)
binary_submission.to_csv('./binary_submission.csv', index=False)
display_df(binary_submission.head())

## Image Level Predictions (Detection Models)

In [ ]:
def format_prediction_string(boxes, scores, labels):
    "generate a single prediction string given boxes, scores and labels"
    pred_strings = []
    if len(boxes) > 0 :
        for j in zip(labels, scores, boxes):
            pred_strings.append("opacity {0:.4f} {1} {2} {3} {4}".format(j[1], j[2][0], j[2][1], j[2][2], j[2][3]))
    else:
        pred_strings.append("none 1 0 0 1 1")
    return " ".join(pred_strings)


def write_submission(test_images_ids, result_boxes, result_scores, result_labels, score_thr):
    "write submissions in pandas dataframe format"
    results = []
    
    for i, image in enumerate(test_images_ids):
        image_id   = test_images_ids[i]
        cur_boxes  = np.array(result_boxes[i])
        cur_scores = np.array(result_scores[i])
        cur_labels = np.array(result_labels[i])

        score_filter = cur_scores >= score_thr
        cur_boxes    = cur_boxes[score_filter]
        cur_scores   = cur_scores[score_filter]
        cur_labels   = cur_labels[score_filter]
        
        result = {'image_id': image_id, 'PredictionString': format_prediction_string(cur_boxes, cur_scores, cur_labels)}
        results.append(result)
    
    return pd.DataFrame(results, columns=['image_id', 'PredictionString'])

## + Efficientnet Models
- D3 (pseudo + ema) (896)
- D5 (ema) (512)

In [ ]:
from ensemble_boxes import *
from effdet import unwrap_bench
from effdet import create_model as create_det
from effdet.config import get_efficientdet_config

In [ ]:
class LoadDetectionDatset(Dataset):
    """Object Detection Dataset"""
    def __init__(self, image_paths, augmentations, img_size):
        self.image_paths = image_paths
        self.augmentations = augmentations
        self.img_size = img_size
        
    def __len__(self): 
        return len(self.image_paths)
    
    def __getitem__(self, index: int):
        image_path = self.image_paths[index]
        image = cv2.cvtColor(cv2.imread(str(image_path)), cv2.COLOR_BGR2RGB)
        image = self.augmentations(image=image)['image']
        
        image_id = str(image_path).split(os.path.sep)[-1].split('.')[0] + '_image'
        ann = dict(img_idx=index, img_size=self.img_size)
        return image, ann, image_id

In [ ]:
@torch.no_grad()        
def make_predictions(dl, net, score_threshold=0.001):
    predictions = {}
    
    for batch in tqdm(dl, dynamic_ncols=True, smoothing=0):
        image, ann, image_id = batch
        images = torch.stack(image).cuda().float()
        
        target = {}
        target['img_scale'] = torch.tensor([1]*images.shape[0]).float().cuda()
        target['img_size']  = torch.tensor([a['img_size'] for a in ann]).cuda().float()

        with torch.no_grad():
            det = net(images, img_info=target)
            for i in range(images.shape[0]):
                boxes  = det[i].detach().cpu().numpy()[:,:4]    
                scores = det[i].detach().cpu().numpy()[:,4]
                indexes= np.where(scores > score_threshold)[0]
                boxes  = boxes[indexes]
                predictions[image_id[i]] = ({'boxes': boxes[indexes], 'scores': scores[indexes]})
    return predictions

In [ ]:
@delegates(weighted_boxes_fusion)
def merge_boxes_from_models(all_models_boxes, all_models_scores, all_models_labels,n_images, **kwargs):
    n_variants = len(all_models_boxes)
    assert len(all_models_scores) == n_variants
    assert len(all_models_labels) == n_variants
    
    print(len(all_models_boxes), len(all_models_scores), len(all_models_labels))

    result_boxes  = []
    result_scores = []
    result_labels = []
    
    for ii in tqdm(range(n_images), dynamic_ncols=True, smoothing=0):
        pred_boxes  = []
        pred_scores = []
        pred_labels = []
        max_value = 10000
        
        for vi in range(n_variants):
            cur_pred_boxes  = np.array(all_models_boxes[vi][ii],  copy=False)
            cur_pred_scores = np.array(all_models_scores[vi][ii], copy=False)
            cur_pred_labels = np.array(all_models_labels[vi][ii], copy=False)
            
            cur_pred_boxes  = cur_pred_boxes / max_value

            pred_boxes.append(cur_pred_boxes)
            pred_scores.append(cur_pred_scores)
            pred_labels.append(cur_pred_labels)

        pred_boxes, pred_scores, pred_labels = weighted_boxes_fusion(
            pred_boxes, pred_scores, pred_labels, **kwargs)
        
        pred_boxes = np.round(pred_boxes * max_value).astype(int)

        assert len(pred_boxes) == len(pred_scores)
        assert len(pred_boxes) == len(pred_scores)
        assert len(pred_boxes) == len(pred_labels)
        
        result_boxes.append(pred_boxes)
        result_scores.append(pred_scores)
        result_labels.append(pred_labels)
    
    return result_boxes, result_scores, result_labels

In [ ]:
imagenames = image_df.image_path.values.tolist()
image_ids  = L(imagenames).map(lambda x: x.split(os.path.sep)[-1].split('.')[0] + '_image')

def collate_fn(batch): return tuple(zip(*batch))

states1 = [
    '../input/psuedo-models-covid/d3_896_stage1_psuedo_ema/model_best_d3_896_stage1_psuedo_fold0_5811.pth.tar',
    '../input/psuedo-models-covid/d3_896_stage1_psuedo_ema/model_best_d3_896_stage1_psuedo_fold1_5814.pth.tar',
    '../input/psuedo-models-covid/d3_896_stage1_psuedo_ema/model_best_d3_896_stage1_psuedo_fold2_6012.pth.tar',
    '../input/psuedo-models-covid/d3_896_stage1_psuedo_ema/model_best_d3_896_stage1_psuedo_fold3_5738.pth.tar',
    '../input/psuedo-models-covid/d3_896_stage1_psuedo_ema/model_best_d3_896_stage1_psuedo_fold4_6182.pth.tar',
]
    
MODEL = 'tf_efficientdet_d3_ap'
base_conf = get_efficientdet_config(MODEL)
SZ = 896
print("Image dims:", SZ)
    
test_transforms = pre_processing(SZ)

ds = LoadDetectionDatset(imagenames, test_transforms, img_size=[SZ, SZ])
dl = DataLoader(ds, batch_size=12, num_workers=2, collate_fn=collate_fn)


predictions = []
for i in tqdm(states1, dynamic_ncols=True):
    bench = create_det(
        model_name=MODEL, bench_task='predict', 
        num_classes=1, bench_labeler=True,
        soft_nms=False, pretrained_backbone=False, 
        image_size = [SZ,SZ])
    
    load_checkpoint(unwrap_bench(bench), i, use_ema=True)
    bench.cuda()
    bench.eval() 
    
    preds = make_predictions(dl, bench)
    predictions.append(preds)
    
    del bench, preds
    gc.collect()
    torch.cuda.empty_cache()
    
del ds, dl
gc.collect()
torch.cuda.empty_cache()

In [ ]:
all_folds_boxes  = []
all_folds_scores = []
all_folds_labels = []

for prediction in tqdm(predictions, smoothing=0, dynamic_ncols=True,):
    pred_boxes = []; pred_scores = []; pred_labels = []

    for image_index in image_ids:
        pred_boxes.append(prediction[image_index]['boxes'].tolist()) 
        pred_scores.append(prediction[image_index]['scores'].tolist())
        pred_labels.append(np.ones(prediction[image_index]['scores'].shape[0]).tolist())
    pred_boxes, pred_scores, pred_labels = np.array(pred_boxes), np.array(pred_scores), np.array(pred_labels)

    for i, image in enumerate(image_ids):
        image_id = image_ids[i]
        image_height, image_width = image_df.loc[image_df.image_id == image_id, ['dim0', 'dim1']].values[0]

        if len(pred_boxes[i]) > 0:
            xscale = image_width / SZ
            yscale = image_height / SZ
            pred_boxes[i][:, [0, 2]] = (pred_boxes[i][:, [0, 2]] * xscale).astype(int)
            pred_boxes[i][:, [1, 3]] = (pred_boxes[i][:, [1, 3]] * yscale).astype(int)

    all_folds_boxes.append(pred_boxes)
    all_folds_scores.append(pred_scores)
    all_folds_labels.append(pred_labels)
    
    
result_boxes, result_scores, result_labels = merge_boxes_from_models(
    all_folds_boxes, all_folds_scores, all_folds_labels, 
    n_images=len(image_ids), weights=None, iou_thr=0.60, conf_type='avg')

d3_preds = write_submission(image_ids, result_boxes, result_scores, result_labels, score_thr=0.0001)
d3_preds.to_csv('efdet2_sub1.csv',index=False)
d3_preds.head()

In [ ]:
imagenames = image_df.image_path.values.tolist()
image_ids  = L(imagenames).map(lambda x: x.split(os.path.sep)[-1].split('.')[0] + '_image')

def collate_fn(batch): return tuple(zip(*batch))

states1 = [
    '../input/covid-models-new/d5_stage1/model_best_d5_fold0_5698.pth.tar',
    '../input/covid-models-new/model_best_d5_fold1_5654_stage2.pth.tar',
    '../input/covid-models-new/model_best_d5_fold2_6040_stage2.pth.tar',
    '../input/covid-models-new/d5_stage1/model_best_d5_fold3_5577.pth.tar',
    '../input/covid-models-new/d5_stage1/model_best_d5_fold4_6100.pth.tar'
]
    
MODEL = 'tf_efficientdet_d5_ap'
base_conf = get_efficientdet_config(MODEL)
SZ = 512
print("Image dims:", SZ)
    
test_transforms = pre_processing(SZ, norm=None)

ds = LoadDetectionDatset(imagenames, test_transforms, img_size=[SZ, SZ])
dl = DataLoader(ds, batch_size=6, collate_fn=collate_fn)

predictions = []
for i in tqdm(states1, dynamic_ncols=True):
    bench = create_det(
        model_name=MODEL, bench_task='predict', 
        num_classes=1, bench_labeler=True,
        soft_nms=False, pretrained_backbone=False, 
        image_size = [SZ,SZ])
    
    load_checkpoint(unwrap_bench(bench), i, use_ema=True)
    bench.cuda()
    bench.eval() 
    
    preds = make_predictions(dl, bench)
    predictions.append(preds)
    
    del bench, preds
    gc.collect()
    torch.cuda.empty_cache()
    
del ds, dl
gc.collect()
torch.cuda.empty_cache()

In [ ]:
all_folds_boxes  = []
all_folds_scores = []
all_folds_labels = []

for prediction in tqdm(predictions, smoothing=0, dynamic_ncols=True,):
    pred_boxes = []; pred_scores = []; pred_labels = []

    for image_index in image_ids:
        pred_boxes.append(prediction[image_index]['boxes'].tolist()) 
        pred_scores.append(prediction[image_index]['scores'].tolist())
        pred_labels.append(np.ones(prediction[image_index]['scores'].shape[0]).tolist())
    pred_boxes, pred_scores, pred_labels = np.array(pred_boxes), np.array(pred_scores), np.array(pred_labels)

    for i, image in enumerate(image_ids):
        image_id = image_ids[i]
        image_height, image_width = image_df.loc[image_df.image_id == image_id, ['dim0', 'dim1']].values[0]

        if len(pred_boxes[i]) > 0:
            xscale = image_width / SZ
            yscale = image_height / SZ
            pred_boxes[i][:, [0, 2]] = (pred_boxes[i][:, [0, 2]] * xscale).astype(int)
            pred_boxes[i][:, [1, 3]] = (pred_boxes[i][:, [1, 3]] * yscale).astype(int)

    all_folds_boxes.append(pred_boxes)
    all_folds_scores.append(pred_scores)
    all_folds_labels.append(pred_labels)
    
    
result_boxes, result_scores, result_labels = merge_boxes_from_models(
    all_folds_boxes, all_folds_scores, all_folds_labels, 
    n_images=len(image_ids), weights=None, iou_thr=0.60, conf_type='avg')

d5_preds = write_submission(image_ids, result_boxes, result_scores, result_labels, score_thr=0.0001)
d5_preds.to_csv('efdet2_sub2.csv',index=False)
d5_preds.head()

## + Yolov5 Models
- yolov5l6_640_psuedo
- yolov5x_640_psuedo

In [ ]:
import sys
sys.path.append('../input/simmyolov5/')

SUB_DIR_1 = '/kaggle/tmp/sub/'
SUB_DIR_2 = '/kaggle/tmp/sub2/'

os.makedirs(SUB_DIR_1, exist_ok=True)
os.makedirs(SUB_DIR_2, exist_ok=True)
os.makedirs('/kaggle/working/subm', exist_ok=True)

In [ ]:
def txt2pandas(Idxs, fold,num,sub):
    image_ids = []
    PredictionStrings = []
    
    for Idx in tqdm(Idxs, dynamic_ncols=True):
        H,W = image_df.loc[image_df.image_id == Idx, ['dim0', 'dim1']].values[0]
        Idx = Idx.split('_')[0]
        txt_label = os.path.join(sub, f'fold{fold}/labels/{Idx}.txt')
        bboxes = []
    
        if os.path.isfile(txt_label):
            in1 = open(txt_label, 'r')
            lines = in1.readlines()
            in1.close()

            for line in lines:
                arr = line.strip().split(' ')
                class_id = 'opacity'
                x = float(arr[1])
                y = float(arr[2])
                w = float(arr[3])
                h = float(arr[4])
                x1 = x - (w / 2)  
                x2 = x + (w / 2)    
                y1 = y - (h / 2)
                y2 = y + (h / 2)
                conf = arr[5]

                x1 = int(round(x1 * W))
                y1 = int(round(y1 * H))
                x2 = int(round(x2 * W))
                y2 = int(round(y2 * H))

                bboxes.append(f"{class_id} {conf} {x1} {y1} {x2} {y2}")
        else:
            print(txt_label)
            class_id = 'none'
            conf = 1.0
            [x1, y1, x2, y2] = [0, 0, 1, 1]
            bboxes.append(f"{class_id} {conf} {x1} {y1} {x2} {y2}")

        image_ids.append(Idx)
        PredictionStrings.append(' '.join(bboxes))
    
    detections = pd.DataFrame()    
    detections['image_id'] = image_ids
    detections['image_id'] = detections['image_id'].map(lambda x: x + '_image')
    detections['PredictionString'] = PredictionStrings
    detections.to_csv(f'/kaggle/working/subm/yolo_fold{fold}{num}.csv', index=False)
    print(f'Dataframe saved to /kaggle/working/subm/yolo_fold{fold}{num}.csv')

In [ ]:
!python /kaggle/input/simmyolov5/detect.py --imgsz 640 --conf-thres 0.001 --iou-thres 0.5 \
        --save-txt --save-conf --exist-ok --nosave \
        --weights ../input/psuedo-models-covid/yolov5l6_640_psuedo/yolov5l6_640_fold0.pt \
        --source {TEST_PATH} --project {SUB_DIR_1} --name fold0

!python /kaggle/input/simmyolov5/detect.py --imgsz 640 --conf-thres 0.001 --iou-thres 0.5 \
        --save-txt --save-conf --exist-ok --nosave \
        --weights ../input/psuedo-models-covid/yolov5l6_640_psuedo/yolov5l6_640_fold1.pt \
        --source {TEST_PATH} --project {SUB_DIR_1} --name fold1

!python /kaggle/input/simmyolov5/detect.py --imgsz 640 --conf-thres 0.001 --iou-thres 0.5 \
        --save-txt --save-conf --exist-ok --nosave \
        --weights ../input/psuedo-models-covid/yolov5l6_640_psuedo/yolov5l6_640_fold2.pt \
        --source {TEST_PATH} --project {SUB_DIR_1} --name fold2

!python /kaggle/input/simmyolov5/detect.py --imgsz 640 --conf-thres 0.001 --iou-thres 0.5 \
        --save-txt --save-conf --exist-ok --nosave \
        --weights ../input/psuedo-models-covid/yolov5l6_640_psuedo/yolov5l6_640_fold3.pt \
        --source {TEST_PATH} --project {SUB_DIR_1} --name fold3

!python /kaggle/input/simmyolov5/detect.py --imgsz 640 --conf-thres 0.001 --iou-thres 0.5 \
        --save-txt --save-conf --exist-ok --nosave \
        --weights ../input/psuedo-models-covid/yolov5l6_640_psuedo/yolov5l6_640_fold4.pt \
        --source {TEST_PATH} --project {SUB_DIR_1} --name fold4


Idxs = image_df.image_id.values.tolist()
for i in range(0, 5):  txt2pandas(Idxs, i,0,SUB_DIR_1)
!rm -r {SUB_DIR_1}

In [ ]:
!python /kaggle/input/simmyolov5/detect.py --imgsz 640 --conf-thres 0.001 --iou-thres 0.5 \
        --save-txt --save-conf --exist-ok --nosave \
        --weights ../input/psuedo-models-covid/yolov5x_640_psuedo/yolov5x_640_fold0.pt  \
        --source {TEST_PATH} --project {SUB_DIR_2} --name fold0

!python /kaggle/input/simmyolov5/detect.py --imgsz 640 --conf-thres 0.001 --iou-thres 0.5 \
        --save-txt --save-conf --exist-ok --nosave \
        --weights ../input/psuedo-models-covid/yolov5x_640_psuedo/yolov5x_640_fold1.pt \
        --source {TEST_PATH} --project {SUB_DIR_2} --name fold1

!python /kaggle/input/simmyolov5/detect.py --imgsz 640 --conf-thres 0.001 --iou-thres 0.5 \
        --save-txt --save-conf --exist-ok --nosave \
        --weights ../input/psuedo-models-covid/yolov5x_640_psuedo/yolov5x_640_fold2.pt \
        --source {TEST_PATH} --project {SUB_DIR_2} --name fold2

!python /kaggle/input/simmyolov5/detect.py --imgsz 640 --conf-thres 0.001 --iou-thres 0.5 \
        --save-txt --save-conf --exist-ok --nosave \
        --weights ../input/psuedo-models-covid/yolov5x_640_psuedo/yolov5x_640_fold3.pt \
        --source {TEST_PATH} --project {SUB_DIR_2} --name fold3

!python /kaggle/input/simmyolov5/detect.py --imgsz 640 --conf-thres 0.001 --iou-thres 0.5 \
        --save-txt --save-conf --exist-ok --nosave \
        --weights ../input/psuedo-models-covid/yolov5x_640_psuedo/yolov5x_640_fold4.pt  \
        --source {TEST_PATH} --project {SUB_DIR_2} --name fold4

for i in range(0, 5): txt2pandas(Idxs, i,1, SUB_DIR_2)  
    
!rm -r {SUB_DIR_2}

In [ ]:
@delegates(weighted_boxes_fusion)
def run_wbf(subm_list, out_path=None, **kwargs):
    max_value = 10000
    preds   = []
    checker = None
    
    for path in subm_list:
        s = pd.read_csv(path)
        s.sort_values('image_id', inplace=True)
        s.reset_index(drop=True, inplace=True)
        ids = s['image_id']
        if checker:
            if tuple(ids) != checker:
                print(set(checker) - set(ids))
                print('Different IDS!', len(tuple(ids)), path)
                exit()
        else:
            checker = tuple(ids)
        preds.append(s['PredictionString'].values)

    if out_path is None:
        out_path = '/kaggle/working/' + 'ensemble_iou_{}.csv'.format(iou_same)
    
    out = open(out_path, 'w')
    
    out.write('image_id,PredictionString\n')
    
    for j, id in enumerate(tqdm(list(checker))):
        
        boxes_list = []
        scores_list = []
        labels_list = []
        empty = True
        for i in range(len(preds)):
            boxes = []
            scores = []
            labels = []
            p1 = preds[i][j]
            if str(p1) != 'nan':
                arr = p1.strip().split(' ')
                for k in range(0, len(arr), 6):
                    cls = 1 if arr[k] == 'opacity' else 0
                    prob = float(arr[k + 1])
                    x1 = float(arr[k + 2]) / max_value
                    y1 = float(arr[k + 3]) / max_value
                    x2 = float(arr[k + 4]) / max_value
                    y2 = float(arr[k + 5]) / max_value
                    boxes.append([x1, y1, x2, y2])
                    scores.append(prob)
                    labels.append(cls)

            boxes_list.append(boxes)
            scores_list.append(scores)
            labels_list.append(labels)

        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, **kwargs)

        if len(boxes) == 0:
            out.write('{},none 1 0 0 1 1\n'.format(id, ))
        else:
            final_str = ''
            for i in range(len(boxes)):
                cls = int(labels[i])
                prob = scores[i]
                x1 = int(boxes[i][0] * max_value)
                y1 = int(boxes[i][1] * max_value)
                x2 = int(boxes[i][2] * max_value)
                y2 = int(boxes[i][3] * max_value)
                if cls == 0:
                    final_str += '{} {} {} {} {} {} '.format('none', prob, 0, 0, 1, 1)
                else:
                    final_str += '{} {} {} {} {} {} '.format('opacity', prob, x1, y1, x2, y2)
            out.write('{},{}\n'.format(id, final_str.strip()))

    out.close()
    print(f"Results ---> {out_path}")
    return out_path

In [ ]:
subm_list1 = [
    '/kaggle/working/subm/yolo_fold00.csv',
    '/kaggle/working/subm/yolo_fold10.csv',
    '/kaggle/working/subm/yolo_fold20.csv',
    '/kaggle/working/subm/yolo_fold30.csv',
    '/kaggle/working/subm/yolo_fold40.csv'
]

subm_list2 = [
    '/kaggle/working/subm/yolo_fold01.csv',
    '/kaggle/working/subm/yolo_fold11.csv',
    '/kaggle/working/subm/yolo_fold21.csv',
    '/kaggle/working/subm/yolo_fold31.csv',
    '/kaggle/working/subm/yolo_fold41.csv'
]


run_wbf(subm_list1, out_path='/kaggle/working/yolo_v5_sub1.csv',  iou_thr=0.60)
run_wbf(subm_list2, out_path='/kaggle/working/yolo_v5_sub2.csv',  iou_thr=0.60)

yolo_sub1 = pd.read_csv('/kaggle/working/yolo_v5_sub1.csv' )
yolo_sub2 = pd.read_csv('/kaggle/working/yolo_v5_sub2.csv' )

## + MMdetection (retinanet_x101_64x4d_fpn)

In [ ]:
import mmdet
import mmcv

from mmcv import Config
from mmcv.runner import load_checkpoint
from mmdet.apis import set_random_seed, inference_detector, \
     init_detector, show_result_pyplot, single_gpu_test
from mmdet.datasets import build_dataset, build_dataloader
from mmdet.models import build_detector

In [ ]:
baseline_cfg_path = "../input/detsiim/retinanet_x101_64x4d_fpn_without_empty/fold0/retinanet_x101_64x4d_fpn_siim_fold0.py"
cfg = Config.fromfile(baseline_cfg_path)

WEIGHTS_FILES = [
    "../input/detsiim/retinanet_x101_64x4d_fpn_without_empty/fold0/best_bbox_mAP_50_epoch_12.pth",
    "../input/detsiim/retinanet_x101_64x4d_fpn_without_empty/fold1/best_bbox_mAP_50_epoch_8.pth",
    "../input/detsiim/retinanet_x101_64x4d_fpn_without_empty/fold2/best_bbox_mAP_50_epoch_10.pth",
    "../input/detsiim/retinanet_x101_64x4d_fpn_without_empty/fold3/best_bbox_mAP_50_epoch_14.pth",
    "../input/detsiim/retinanet_x101_64x4d_fpn_without_empty/fold4/best_bbox_mAP_50_epoch_7.pth",
]

assert [os.path.isfile(w) for w in WEIGHTS_FILES]

In [ ]:
score_threshold = cfg.model.test_cfg.score_thr

def format_pred(boxes: np.ndarray, scores: np.ndarray, labels: np.ndarray) -> str:
    pred_strings = []
    for label, score, bbox in zip(labels, scores, boxes):
        xmin, ymin, xmax, ymax = bbox.astype(np.int64)
        pred_strings.append(f"opacity {score:.16f} {xmin} {ymin} {xmax} {ymax}")
    return " ".join(pred_strings)

for i, weights in enumerate(tqdm(WEIGHTS_FILES,  dynamic_ncols=True)):
    results = []
    model = init_detector(cfg, weights, device='cuda:0')
    model.eval()

    with torch.no_grad():
        for index, row in tqdm(image_df.iterrows(), total=image_df.shape[0], dynamic_ncols=True):
            original_H, original_W = (int(row.dim0), int(row.dim1))
            predictions = inference_detector(model, row.image_path)
            boxes, scores, labels = (list(), list(), list())

            for k, cls_result in enumerate(predictions):
                if cls_result.size != 0:
                    if len(labels) == 0:
                        boxes  = np.array(cls_result[:, :4])
                        scores = np.array(cls_result[:, 4])
                        labels = np.array([k]*len(cls_result[:, 4]))
                    else:    
                        boxes  = np.concatenate((boxes, np.array(cls_result[:, :4])))
                        scores = np.concatenate((scores, np.array(cls_result[:, 4])))
                        labels = np.concatenate((labels, [k]*len(cls_result[:, 4])))
                    
            indexes = np.where(scores > score_threshold)
            boxes   = boxes[indexes]
            scores  = scores[indexes]
            labels  = labels[indexes]
            
            IMAGE_DIMS = cv2.imread(row.image_path).shape[:2]

            if len(labels) != 0:
                h_ratio = original_H/IMAGE_DIMS[0]
                w_ratio = original_W/IMAGE_DIMS[1]
                boxes[:, [0, 2]] *= w_ratio
                boxes[:, [1, 3]] *= h_ratio
                result = {"image_id": row.image_id, "PredictionString": format_pred(boxes, scores, labels)}
                results.append(result)
    del model
    torch.cuda.empty_cache()
    gc.collect()
    detection_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
    detection_df.to_csv(f'retinanet_x101_64x4d_fpn_fold_{i}.csv', index=False)

In [ ]:
retina_subs = [
    '/kaggle/working/retinanet_x101_64x4d_fpn_fold_0.csv',
    '/kaggle/working/retinanet_x101_64x4d_fpn_fold_1.csv',
    '/kaggle/working/retinanet_x101_64x4d_fpn_fold_2.csv',
    '/kaggle/working/retinanet_x101_64x4d_fpn_fold_3.csv',
    '/kaggle/working/retinanet_x101_64x4d_fpn_fold_4.csv',
]


run_wbf(retina_subs, out_path='/kaggle/working/retinanet_x101_64x4d_fpn_sub.csv',  iou_thr=0.60)
retinanet_sub = pd.read_csv('/kaggle/working/retinanet_x101_64x4d_fpn_sub.csv')

## Merge all Image Level (Detection) Predictions

In [ ]:
@delegates(weighted_boxes_fusion)
def merge_boxes_from_models(test_files, test_filenames, **kwargs):
    n_images   = len(test_filenames)
    n_variants = len(test_files)

    result_boxes = []
    result_scores = []
    result_labels = []
    for ii in tqdm(range(n_images), dynamic_ncols=True):
        image_id = test_filenames[ii]
        pred_variants = pred_locations[image_id]
        assert len(pred_variants) == n_variants

        pred_boxes = []
        pred_scores = []
        pred_labels = []
        max_value = 10000
        for vi in range(n_variants):
            if len(pred_variants[vi]) > 0:
                cur_pred_labels, cur_pred_scores, cur_pred_boxes = list(zip(*pred_variants[vi]))
            else:
                cur_pred_labels = []
                cur_pred_scores = []
                cur_pred_boxes = []
            cur_pred_boxes = np.array(cur_pred_boxes, copy=False)
            cur_pred_scores = np.array(cur_pred_scores, copy=False)
            cur_pred_labels = np.array(cur_pred_labels, copy=False)

            # WBF expects the coordinates in 0-1 range.
            cur_pred_boxes = cur_pred_boxes / max_value

            pred_boxes.append(cur_pred_boxes)
            pred_scores.append(cur_pred_scores)
            pred_labels.append(cur_pred_labels)

        pred_boxes, pred_scores, pred_labels = weighted_boxes_fusion(pred_boxes, pred_scores, pred_labels, **kwargs)
        pred_boxes = np.round(pred_boxes * max_value).astype(int)

        assert len(pred_boxes) == len(pred_scores)
        assert len(pred_boxes) == len(pred_scores)
        assert len(pred_boxes) == len(pred_labels)
        result_boxes.append(pred_boxes)
        result_scores.append(pred_scores)
        result_labels.append(pred_labels)

    return result_boxes, result_scores, result_labels

In [ ]:
test_files = [
    './efdet2_sub1.csv', 
    './efdet2_sub2.csv',  
    './yolo_v5_sub1.csv', 
    './yolo_v5_sub2.csv', 
    './retinanet_x101_64x4d_fpn_sub.csv'
]

print('# models', len(test_files))
print(test_files)

pred_locations = {}

for model_index in range(len(test_files)):
    # load table
    with open(test_files[model_index], mode='r') as infile:
        # open reader
        reader = csv.reader(infile)
        # skip header
        next(reader, None)
        # loop through rows
        for rows in reader:
            # retrieve information
            filename = rows[0]
            # print(rows[0])
            parts = rows[1].split()
            # print(parts)
            assert len(parts) % 6 == 0
            locations = []
            for ind in range(len(parts) // 6):
                #label = int(parts[ind * 6])
                label = parts[ind * 6]
                label = 1 if label == 'opacity' else 0
                score = float(parts[ind * 6 + 1])
                location = int(float(parts[ind * 6 + 2])), int(float(parts[ind * 6 + 3])), \
                           int(float(parts[ind * 6 + 4])), int(float(parts[ind * 6 + 5]))
                if score > 0 and label < 14:
                    locations.append((label, score, location))
            if filename in pred_locations:
                pred_locations[filename].append(locations)
            else:
                pred_locations[filename] = [locations]
                
                
test_filenames = [*pred_locations]
test_filenames.sort()

result_boxes, result_scores, result_labels = merge_boxes_from_models(
    test_files, test_filenames, weights=[0.22,0.23,0.25,0.2,0.1], 
    iou_thr=0.625, conf_type='avg', allows_overflow=True)

## Submission for opactity only

In [ ]:
image_df = write_submission(test_filenames, result_boxes, result_scores, result_labels, 0)
image_df.head()

## Add Binary None Predictions to Image Level (Opacity) Predictions

In [ ]:
image_preds_df = pd.merge(image_df, binary_submission, on='image_id')
image_preds_df = image_preds_df.drop(columns=['study_id'])
image_preds_df.columns = ['id', 'PredictionString', 'none']
image_preds_df.head()

In [ ]:
for i in tqdm(range(0,image_preds_df.shape[0])):
    if image_preds_df.loc[i,'PredictionString'] == "none 1 0 0 1 1":
        image_preds_df.loc[i,'PredictionString']='none 1 0 0 1 1'
        continue
    
    image_split = image_preds_df.loc[i,'PredictionString'].split()
    binary_none = image_preds_df.loc[i,'none']
    image_df_list = []
    preds_for_none = 0
    final_image = []
    
    for j in range(int(len(image_split) / 6)):
        if j==0:
            preds_for_none = 1 - ((float(image_split[6 * j + 1])**0.20)*(1-binary_none)**0.80)
        else:
            preds_for_none = preds_for_none

        image_df_list.append([
            'opacity',
            (float(image_split[6 * j + 1])**0.80)*(1-binary_none)**0.20,
            image_split[6 * j + 2],
            image_split[6 * j + 3],
            image_split[6 * j + 4],
            image_split[6 * j + 5]
        ])
    image_df_list = sorted(image_df_list, key = lambda x: x[1], reverse=True)
    for k in range(len(image_df_list)):
        image_df_list[k][1] = str(image_df_list[k][1])
        final_image.extend(image_df_list[k])

    image_preds_df.loc[i,'PredictionString'] = ' '.join(final_image)

    image_preds_df.loc[i,'PredictionString'] = image_preds_df.loc[i,'PredictionString'] + ' none ' + \
    str(preds_for_none) + ' 0 0 1 1'
    image_preds_df.loc[i,'none'] = preds_for_none

In [ ]:
image_df = image_preds_df[['id','PredictionString']].copy()
image_df.head()

## Generate Final Submission

In [ ]:
submission = pd.concat([study_df[['id','PredictionString']],image_df],axis=0)
submission = submission[['id','PredictionString']]
submission.head()

In [ ]:
submission.tail()

In [ ]:
!rm -r /kaggle/working/
!rm -r /kaggle/tmp/

In [ ]:
submission.to_csv('submission.csv',index=False)